## Image Encodings

In [149]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader, IterableDataset

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import time
import os
import copy
import json

from PIL import Image

from transformers import BertTokenizer, BertModel
import pickle

# making sure that the whole embedding tensor is printed in output
torch.set_printoptions(threshold=10_000)

In [2]:
# making sure the feature extraction runs on GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [70]:
# class MyDataset(torch.utils.data.Dataset):
#     def __init__(self, dir_path, file_path, transforms):
#         self.dir = dir_path
#         self.file_path = file_path
# #         self.img_id_mapping = {}
#         self.image_paths = []
#         self.transforms = transforms

#         test_file = open(file_path)
#         test_data = json.load(test_file)
#         test_file.close()

#         for row in test_data:
#             id = row["id"]
#             for image_path in row["images"]:
#                 self.image_paths.append((id, image_path))
# #                 self.img_id_mapping[image_path] = id
                
#     def __len__(self):
#         return len(self.image_paths)
    
#     def __getitem__(self, idx):
#         image_path = self.dir + self.image_paths[idx][1][0] + "/" + self.image_paths[idx][1][1] + "/" + self.image_paths[idx][1][2] + "/" + self.image_paths[idx][1][3] + "/" + self.image_paths[idx][1]
#         img = Image.open(image_path)
#         img = self.transforms(img)
        
#         return self.image_paths[idx][0], img, self.image_paths[idx][1]

In [71]:
model_conv = torchvision.models.resnet50(pretrained=True)
last_layer = model_conv._modules.get('avgpool')

# transforming each image
data_transforms = transforms.Compose([
        transforms.Resize((256,256)),
        transforms.CenterCrop((224,224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

# test_dataset = MyDataset("D:/Projects/ML Project/test/", "E:/data/test/image.json", data_transforms)
# test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

In [72]:
"""Function that hooks to the layer before the FCN in ResNet and extracts the output embedding from it"""

def get_vector(img):
    my_embedding = torch.zeros(2048)
    
    def copy_data(m, i, o):
        my_embedding.copy_(o.data.reshape(o.data.size(1)))

    h = last_layer.register_forward_hook(copy_data)
    model_conv(img)
    h.remove()

    return my_embedding

In [ ]:
# output = []
# for i, (id, img, image_name) in enumerate(test_loader):
#     emb = get_vector(img[i % 16].unsqueeze(0))
#     output.append((id, image_name, emb))
#     if len(output) % 1000 == 0:
#         print(i)

In [ ]:
# torch.save(output, "test_emb.pt")

In [ ]:
# file containing preprocessed image and text data
# file path to be changed for different embeddings

test_file = open('E:/data/test/image.json')
test_data = json.load(test_file)
test_file.close()
output = []

# feature extraction logic
for row in test_data:
    id = row["id"]
    for image in row["images"]:
        image_path = "D:/Projects/ML Project/test/" + image[0] + "/" + image[1] + "/" + image[2] + "/" + image[3] + "/" + image
        img = Image.open(image_path)

        transformed_image = data_transforms(img)
        emb = get_vector(transformed_image.unsqueeze(0))    
        
        # store embedding as (id, image_file_name, embedding)
        output.append((id, image, emb))
        
        if len(output) % 1000 == 0:
            print(len(output))

In [79]:
torch.save(output, "E:/test_emb.pt")

## Text Encodings

In [145]:
# CHANGE FILENAME HERE
test_file = open('E:/data/train/text.json')
test_data = json.load(test_file)
test_file.close()

In [156]:
class JsonDataset(IterableDataset):
    def __init__(self, file):
        self.file = file

    def __iter__(self):
        for row in self.file:
            id = row["id"]
            title = row["title"]
            ingredients = row["ingredients"]
            instructions = row["instructions"]

            ingredient_text = ""
            instructions_text = ""

            ingredient_text = " ".join(ingredient["text"] for ingredient in ingredients)
            instructions_text = " ".join(instruction["text"] for instruction in instructions)
            
            full_text = title + " " + ingredient_text + " " + instructions_text

            yield id, title, ingredient_text, instructions_text, full_text

    
dataset = JsonDataset(test_data)
dataloader = DataLoader(dataset, batch_size=1)

In [151]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
counter = 0

# in case error occurs, a range of rows to start from
# batch_start = 
# batch_end = 100000

for (id, title, ingredient_text, instructions_text, full_text) in dataloader:
    
#     if (counter in range(batch_start, batch_end)):
    # encode title
    encoded_title = tokenizer(title, return_tensors='pt', max_length=512, truncation = True)
    output_title = model(**encoded_title)
    output_title = torch.mean(output_title["last_hidden_state"], dim = 1)

    with open('E:/train_text_title_6.pkl', 'ab') as f:
        pickle.dump((id, output_title.squeeze(0)), f)

    # encode ingredients
    encoded_ingredients = tokenizer(ingredient_text, return_tensors='pt', max_length=512, truncation = True)
    output_ingredients = model(**encoded_ingredients)
    output_ingredients = torch.mean(output_ingredients["last_hidden_state"], dim = 1)

    with open('E:/train_text_ingredients_6.pkl', 'ab') as f:
        pickle.dump((id, output_ingredients.squeeze(0)), f)

    # encode instructions
    encoded_instructions = tokenizer(instructions_text, return_tensors='pt', max_length=512, truncation = True)
    output_instructions = model(**encoded_instructions)
    output_instructions = torch.mean(output_instructions["last_hidden_state"], dim = 1)

    with open('E:/train_text_instructions_6.pkl', 'ab') as f:
        pickle.dump((id, output_instructions.squeeze(0)), f)

    #encode everything
    encoded_full = tokenizer(full_text, return_tensors='pt', max_length=512, truncation = True)
    output_full = model(**encoded_full)
    output_full = torch.mean(output_full["last_hidden_state"], dim = 1)

    with open('E:/train_text_full_6.pkl', 'ab') as f:
        pickle.dump((id, output_full.squeeze(0)), f)

    print(counter)
    counter += 1


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
# check if encodings generated properly
emb = []
with open('E:/train_text_title_2.pkl', 'rb') as f:
    while True:
        emb.append(pickle.load(f))

## Aligning Text Encodings

In [134]:
img_train = torch.load("E:/train_emb.pt")

In [135]:
img_val = torch.load("E:/val_emb.pt")

In [136]:
img_test = torch.load("E:/test_emb.pt")

In [46]:
text_test_full = []
with open('E:/test_text_full.pkl', 'rb') as f:
    try:
        while True:
            text_test_full.append(pickle.load(f))
    except:
        pass

text_test_ingredients = []
with open('E:/test_text_ingredients.pkl', 'rb') as f:
    try:
        while True:
            text_test_ingredients.append(pickle.load(f))
    except:
        pass

text_test_instructions = []
with open('E:/test_text_instructions.pkl', 'rb') as f:
    try:
        while True:
            text_test_instructions.append(pickle.load(f))
    except:
        pass

text_test_title = []
with open('E:/test_text_title.pkl', 'rb') as f:
    try:
        while True:
            text_test_title.append(pickle.load(f))
    except:
        pass

In [66]:
text_val_title = []
with open('E:/val_text_title.pkl', 'rb') as f:
    try:
        while True:
            text_val_title.append(pickle.load(f))
    except:
        pass

text_val_instructions = []
with open('E:/val_text_instructions.pkl', 'rb') as f:
    try:
        while True:
            text_val_instructions.append(pickle.load(f))
    except:
        pass

text_val_ingredients = []
with open('E:/val_text_ingredients.pkl', 'rb') as f:
    try:
        while True:
            text_val_ingredients.append(pickle.load(f))
    except:
        pass

text_val_full = []
with open('E:/val_text_full.pkl', 'rb') as f:
    try:
        while True:
            text_val_full.append(pickle.load(f))
    except:
        pass

### Test Encodings

In [53]:
i, j = 0, 0
text_test_full_final = []
text_test_ingredients_final = []
text_test_instructions_final = []
text_test_title_final = []

while i < len(img_test):
    id = img_test[i][0]

    if text_test_full[j][0][0] == id and text_test_ingredients[j][0][0] == id and \
    text_test_instructions[j][0][0] == id and text_test_title[j][0][0] == id:
        text_test_full_final.append(text_test_full[j][1])
        text_test_ingredients_final.append(text_test_ingredients[j][1])
        text_test_instructions_final.append(text_test_instructions[j][1])
        text_test_title_final.append(text_test_title[j][1])

        i += 1
    else:
        j += 1

In [64]:
with open('E:/text_test_full_final.pkl', 'wb') as f:
    pickle.dump(text_test_full_final ,f)
with open('E:/text_test_instructions_final.pkl', 'wb') as f:
    pickle.dump(text_test_instructions_final ,f)
with open('E:/text_test_ingredients_final.pkl', 'wb') as f:
    pickle.dump(text_test_ingredients_final ,f)
with open('E:/text_test_title_final.pkl', 'wb') as f:
    pickle.dump(text_test_title_final ,f)

### Validation Encodings

In [85]:
len(text_val_title)

60422

In [73]:
len(img_val)

133842

In [ ]:
i, j = 0, 0
# text_val_full_final = []
text_val_ingredients_final = []
text_val_instructions_final = []
text_val_title_final = []

while i < len(img_val):
    id = img_val[i][0]
#     print(id, text_val_full[j][0][0])
    if text_val_ingredients[j][0][0] == id and \
    text_val_instructions[j][0][0] == id and text_val_title[j][0][0] == id:
#         text_val_full_final.append(text_val_full[j][1])
        text_val_ingredients_final.append(text_val_ingredients[j][1])
        text_val_instructions_final.append(text_val_instructions[j][1])
        text_val_title_final.append(text_val_title[j][1])

        i += 1
    else:
        j += 1

In [87]:
len(text_val_ingredients_final)

133842

In [88]:
# with open('E:/text_val_full_final.pkl', 'wb') as f:
#     pickle.dump(text_val_full_final ,f)
with open('E:/text_val_instructions_final.pkl', 'wb') as f:
    pickle.dump(text_val_instructions_final ,f)
with open('E:/text_val_ingredients_final.pkl', 'wb') as f:
    pickle.dump(text_val_ingredients_final ,f)
with open('E:/text_val_title_final.pkl', 'wb') as f:
    pickle.dump(text_val_title_final ,f)

## Train Encodings

In [90]:
text_train_title = []
with open('E:/train_text_title_1.pkl', 'rb') as f:
    try:
        while True:
            text_train_title.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_title2.pkl', 'rb') as f:
    try:
        while True:
            text_train_title.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_title_3.pkl', 'rb') as f:
    try:
        while True:
            text_train_title.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_title_200000.pkl', 'rb') as f:
    try:
        while True:
            text_train_title.append(pickle.load(f))
    except:
        pass
    
text_train_ingredients = []
with open('E:/train_text_ingredients_1.pkl', 'rb') as f:
    try:
        while True:
            text_train_ingredients.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_ingredients2.pkl', 'rb') as f:
    try:
        while True:
            text_train_ingredients.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_ingredients_3.pkl', 'rb') as f:
    try:
        while True:
            text_train_ingredients.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_ingredients_200000.pkl', 'rb') as f:
    try:
        while True:
            text_train_ingredients.append(pickle.load(f))
    except:
        pass
    
text_train_instructions = []
with open('E:/train_text_instructions_1.pkl', 'rb') as f:
    try:
        while True:
            text_train_instructions.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_instructions2.pkl', 'rb') as f:
    try:
        while True:
            text_train_instructions.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_instructions_3.pkl', 'rb') as f:
    try:
        while True:
            text_train_instructions.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_instructions_200000.pkl', 'rb') as f:
    try:
        while True:
            text_train_instructions.append(pickle.load(f))
    except:
        pass

text_train_full = []
with open('E:/train_text_full_1.pkl', 'rb') as f:
    try:
        while True:
            text_train_full.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_full2.pkl', 'rb') as f:
    try:
        while True:
            text_train_full.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_full_3.pkl', 'rb') as f:
    try:
        while True:
            text_train_full.append(pickle.load(f))
    except:
        pass
with open('E:/train_text_full_200000.pkl', 'rb') as f:
    try:
        while True:
            text_train_full.append(pickle.load(f))
    except:
        pass  

In [ ]:
i, j = 0, 0
text_train_full_final = []
text_train_ingredients_final = []
text_train_instructions_final = []
text_train_title_final = []

while i < len(img_train):
    id = img_train[i][0]
#     print(id, text_train_full[j][0][0])
    if text_train_ingredients[j][0][0] == id and \
    text_train_instructions[j][0][0] == id and text_train_title[j][0][0] == id:
        text_train_full_final.append(text_train_full[j][1])
        text_train_ingredients_final.append(text_train_ingredients[j][1])
        text_train_instructions_final.append(text_train_instructions[j][1])
        text_train_title_final.append(text_train_title[j][1])

        i += 1
    else:
        j += 1

## Formatting Image Encodings

In [142]:
img_train_final = [img_train[i][2] for i in range(len(img_train))]
img_val_final = [img_val[i][2] for i in range(len(img_val))]
img_test_final = [img_test[i][2] for i in range(len(img_test))]

In [144]:
torch.save(img_train_final, "E:/img_train_final.pt")
torch.save(img_val_final, "E:/img_val_final.pt")
torch.save(img_test_final, "E:/img_test_final.pt")

## Loading Encodings